In [ ]:
##########################################
# MLX90640 Thermal Camera w Raspberry Pi #
##########################################

import time, datetime
import board
import busio
import numpy as np
import adafruit_mlx90640
import matplotlib.pyplot as plt
from IPython import display


i2c = busio.I2C(board.SCL, board.SDA)

mlx = adafruit_mlx90640.MLX90640(i2c)
# the refresh should be set for fps * 2
mlx.refresh_rate = adafruit_mlx90640.RefreshRate.REFRESH_16_HZ

local_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
print(local_time)
_zero = _start = time.time()

frame = np.zeros(768)

while True:
    try:
        _start = time.time()
        mlx.getFrame(frame)
    except ValueError as e:
        print(e)
        continue

    res = np.array(frame).reshape((24,32))
    res = np.around(res, 2)
    seconds = time.time() - _start
    fps = 1 / seconds
    label = time.time() - _zero
    print('{} FPS: {}'.format(label, round(fps, 1)))
#     plt.title('{} FPS: {}'.format(label, round(fps, 1)))
#     plt.imshow(res)
#     plt.show()
#     display.clear_output(wait=True)